## FAST-TEXT

In [ ]:
from fasttext import train_supervised 
import pandas as pd
import csv
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
df_train_neg = pd.read_csv("data/twitter-datasets/train_neg.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)
df_train_pos = pd.read_csv("data/twitter-datasets/train_pos.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)
df_train_neg["label"] = "__label__0" # Need the __class__ prefix for fast-text model specs
df_train_pos["label"] = "__label__1"
df_train = pd.concat([df_train_pos,df_train_neg])
df_train

In [ ]:
df_test = pd.read_csv("data/twitter-datasets/test_data.txt", delimiter="\t", header=None, names = ['tweets'], error_bad_lines=False)
#df_test["tweets"] = df_test["tweets"].apply(lambda row: row.split(",",2)[1])

In [ ]:
df_test

In [ ]:
def remove_tags(df):
    df_cleaned = df.copy()
    df_cleaned['tweets'] = df_cleaned['tweets'].apply(lambda tweet: re.sub(r'<.*?>', '', tweet).strip())
    return df_cleaned

def tokenize_and_preprocess(df, stop_words = False, stemming = False, lemmatization = False):
    df_cleaned = df.copy()
    df_cleaned['tokens'] = df_cleaned['tweets'].apply(lambda tweet: word_tokenize(tweet))
    # remove stop words
    if stop_words:
        stop_words = stopwords.words('english')
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [token for token in tokens if token.lower() not in stop_words])
    # stemming
    if stemming:
        ps = PorterStemmer()
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [ps.stem(token) for token in tokens])
    # lemmatization
    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        df_cleaned['tokens'] = df_cleaned['tokens'].apply(lambda tokens: [wordnet_lemmatizer.lemmatize(token) for token in tokens])
    # remove the tweets columns
    df_cleaned.drop(['tweets'], axis=1, inplace=True)
    df_cleaned = df_cleaned.reindex(columns=['tokens', 'label'])
    #df_cleaned["tweets"] = df_cleaned["tokens"].apply(lambda tokens: " ".join(tokens))
    return df_cleaned

In [ ]:
#df_train = tokenize_and_preprocess(remove_tags(df_train),stop_words=True,stemming=True,lemmatization=True)

In [ ]:
df_train = remove_tags(df_train)

In [ ]:
df_train

In [ ]:
#df_test = tokenize_and_preprocess(remove_tags(df_test),stop_words=True,stemming=True,lemmatization=True)
df_test = remove_tags(df_test)

In [ ]:
df_test

In [ ]:
train_file = 'data/fasstext_train.txt'
df_train.to_csv(train_file, header=None, index=False, sep=' ', columns=["label","tweets"])
test_file =  'data/fasstext_test.txt'
df_test.to_csv(test_file, header=None, index=False, sep=' ', columns=["tweets"])

In [ ]:
model = train_supervised(input=train_file)

In [ ]:
print('vocab size: ', len(model.words))

In [ ]:
print('label size: ', len(model.labels))

In [ ]:
predictions = []
file1 = open('data/fasstext_test.txt', 'r')
lines = file1.readlines()
for line in lines:
    pred = model.predict(line.strip())[0][0]
    if pred == "__label__0":
        predictions.append(-1)
    else:    
        predictions.append(1)

In [ ]:
predictions

In [ ]:
ids=[i for i in range(1,len(predictions)+1)]
with open("data/submission.csv", 'w', newline='') as csvfile:
    fieldnames = ['Id', 'Prediction']
    writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
    writer.writeheader()
    for r1, r2 in zip(ids, predictions):
        writer.writerow({'Id':int(r1),'Prediction':int(r2)})